In [1]:
%%bash --bg
roscore

Starting job # 0 in a separate thread.


In [2]:
%%bash --bg
rosrun stage_ros stageros -g $(rospack find stage_ros)/world/willow-erratic.world

Starting job # 2 in a separate thread.


In [13]:
%%bash
rosparam set use_sim_time true

In [17]:
!rostopic list

/base_pose_ground_truth
/base_scan
/clock
/cmd_vel
/odom
/rosout
/rosout_agg
/tf


In [18]:
!rostopic info /clock

Type: rosgraph_msgs/Clock

Publishers: 
 * /stageros (http://jupyter-robinlabuji-2dros-2dstage-2dklaw0bn9:35435/)

Subscribers: None




In [19]:
!rostopic echo -n 1 /base_pose_ground_truth

header: 
  seq: 2873
  stamp: 
    secs: 287
    nsecs: 400000000
  frame_id: "odom"
child_frame_id: ''
pose: 
  pose: 
    position: 
      x: -11.277
      y: 23.266
      z: 0.0
    orientation: 
      x: 0.0
      y: 0.0
      z: 1.0
      w: 6.12323399574e-17
  covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
twist: 
  twist: 
    linear: 
      x: 0.0
      y: 0.0
      z: 0.0
    angular: 
      x: 0.0
      y: 0.0
      z: 0.0
  covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---


In [16]:
!rostopic echo -n 1 /clock

clock: 
  secs: 207
  nsecs: 100000000
---
